## Imports

In [1]:
import h5py
import os
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

## Funciones

In [2]:
def print_structure(name, obj):
    """Callback function to print the structure of the HDF5 file."""
    print(name)
    if isinstance(obj, h5py.Group):
        print("  [Group]")
    elif isinstance(obj, h5py.Dataset):
        print(f"  [Dataset] Shape: {obj.shape}, Data Type: {obj.dtype}")

def plot_ozone_map(ozone_data):
    """Plots the ozone data on a global map."""
    # Handle missing values (replace -9999 or similar with NaN)
    ozone_data = np.where(ozone_data < 0, np.nan, ozone_data)

    # Create latitude and longitude grids
    lons = np.linspace(-180, 180, ozone_data.shape[1])  # Longitudes
    lats = np.linspace(-90, 90, ozone_data.shape[0])    # Latitudes
    lon_grid, lat_grid = np.meshgrid(lons, lats)

    # Create the map
    fig = plt.figure(figsize=(12, 6))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_title("Ozone Levels (ColumnAmountO3)", fontsize=14)

    # Add features
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.add_feature(cfeature.LAND, edgecolor="black", facecolor="lightgray")
    ax.set_global()

    # Plot the ozone data
    c = ax.pcolormesh(lon_grid, lat_grid, ozone_data, shading="auto", cmap="viridis", transform=ccrs.PlateCarree())
    
    # Add a colorbar
    cb = plt.colorbar(c, orientation="horizontal", pad=0.05)
    cb.set_label("Ozone Level (DU)")  # Adjust based on actual units

    plt.show()

def plot_ozone_map_uruguay(ozone_data):
    """Plots the ozone data, zoomed in on Uruguay."""
    # Handle missing values (replace -9999 or similar with NaN)
    ozone_data = np.where(ozone_data < 0, np.nan, ozone_data)

    # Create latitude and longitude grids
    lons = np.linspace(-180, 180, ozone_data.shape[1])  # Longitudes
    lats = np.linspace(-90, 90, ozone_data.shape[0])    # Latitudes
    lon_grid, lat_grid = np.meshgrid(lons, lats)

    # Create the map
    fig = plt.figure(figsize=(10, 6))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_title("Ozone Levels in Uruguay (ColumnAmountO3)", fontsize=14)

    # Add features
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.add_feature(cfeature.LAND, edgecolor="black", facecolor="lightgray")

    # Zoom into Uruguay
    ax.set_extent([-58, -53, -35, -30], crs=ccrs.PlateCarree())

    # Plot the ozone data
    c = ax.pcolormesh(lon_grid, lat_grid, ozone_data, shading="auto", cmap="viridis", transform=ccrs.PlateCarree())
    
    # Add a colorbar
    cb = plt.colorbar(c, orientation="horizontal", pad=0.05)
    cb.set_label("Ozone Level (DU)")  # Adjust based on actual units

    plt.show()

In [3]:
def print_uruguay_data(ozone_data):
    """Prints ozone data values over Uruguay."""
    # Create latitude and longitude arrays
    lons = np.linspace(-180, 180, ozone_data.shape[1])  # Longitudes
    lats = np.linspace(-90, 90, ozone_data.shape[0])    # Latitudes

    # Define Uruguay's bounds
    lat_min, lat_max = -35, -30
    lon_min, lon_max = -58, -53

    # Find indices for Uruguay
    lat_indices = np.where((lats >= lat_min) & (lats <= lat_max))[0]
    lon_indices = np.where((lons >= lon_min) & (lons <= lon_max))[0]

    # Extract data for Uruguay
    uruguay_data = ozone_data[np.ix_(lat_indices, lon_indices)]

    # Print the data
    print(f"Ozone data for Uruguay (lat: {lat_min} to {lat_max}, lon: {lon_min} to {lon_max}):")
    print(uruguay_data)

    # Handle missing or invalid data
    if np.all(np.isnan(uruguay_data)):
        print("No valid data available for Uruguay in this dataset.")
    else:
        print("Valid data range:", np.nanmin(uruguay_data), "to", np.nanmax(uruguay_data))


In [11]:
def analyze_ozone_in_uruguay(ozone_data, file):
    """Finds the lowest, highest ozone levels and ozone levels ≤ 220 in Uruguay."""
    # Handle missing or invalid values (e.g., -9999)
    ozone_data = np.where(ozone_data < 0, np.nan, ozone_data)

    # Create latitude and longitude arrays
    lons = np.linspace(-180, 180, ozone_data.shape[1])  # Longitudes
    lats = np.linspace(-90, 90, ozone_data.shape[0])    # Latitudes

    # Define Uruguay's bounds
    lat_min, lat_max = -35, -30
    lon_min, lon_max = -58, -53

    # Find indices for Uruguay
    lat_indices = np.where((lats >= lat_min) & (lats <= lat_max))[0]
    lon_indices = np.where((lons >= lon_min) & (lons <= lon_max))[0]

    # Subset the data for Uruguay
    uruguay_data = ozone_data[np.ix_(lat_indices, lon_indices)]

    # Find the minimum and maximum ozone levels in Uruguay
    min_ozone_value = np.nanmin(uruguay_data)
    max_ozone_value = np.nanmax(uruguay_data)

    # Find the locations (indices) of the min and max values
    min_indices = np.where(uruguay_data == min_ozone_value)
    max_indices = np.where(uruguay_data == max_ozone_value)

    # Convert indices to latitudes and longitudes
    min_lat = lats[lat_indices][min_indices[0][0]]
    min_lon = lons[lon_indices][min_indices[1][0]]
    max_lat = lats[lat_indices][max_indices[0][0]]
    max_lon = lons[lon_indices][max_indices[1][0]]

    # Print the minimum and maximum values
    # print(f"Lowest ozone level in Uruguay: {min_ozone_value:.2f}")
    # print(f"  Latitude: {min_lat:.2f}, Longitude: {min_lon:.2f}")
    # print(f"Highest ozone level in Uruguay: {max_ozone_value:.2f}")
    # print(f"  Latitude: {max_lat:.2f}, Longitude: {max_lon:.2f}")

    # Find indices within Uruguay where ozone level is ≤ 220
    low_ozone_indices = np.where(uruguay_data <= 220)

    # Map these indices back to latitude and longitude
    if low_ozone_indices[0].size == 0:
        pass
        # print("No data points with ozone levels ≤ 220 found in Uruguay.")
    else:
        latitudes = lats[lat_indices][low_ozone_indices[0]]
        longitudes = lons[lon_indices][low_ozone_indices[1]]
        ozone_values = uruguay_data[low_ozone_indices]
            
        # Print results for ozone ≤ 220
        print(file)
        print(f"Found {len(latitudes)} data points in Uruguay with ozone levels ≤ 220:")
        for lat, lon, val in zip(latitudes, longitudes, ozone_values):
            print(f"Latitude: {lat:.2f}, Longitude: {lon:.2f}, Ozone Level: {val:.2f}")
        
        print("")

## Busqueda bajo ozono

In [12]:
for file_name in os.listdir('OMDOAO3e'):
    file_path = f"OMDOAO3e/{file_name}"
    dataset_path = "HDFEOS/GRIDS/ColumnAmountO3/Data Fields/ColumnAmountO3"
    try:
        # Read the ozone data
        with h5py.File(file_path, "r") as file:
            ozone_data = file[dataset_path][:]
            #print(f"Ozone data loaded with shape: {ozone_data.shape}")
    
        # Plot the ozone levels
        analyze_ozone_in_uruguay(ozone_data, file_name)
    
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except KeyError:
        print(f"Dataset not found: {dataset_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

OMI-Aura_L3-OMDOAO3e_2024m0111_v003-2024m0113t021446.he5
Found 1 data points in Uruguay with ozone levels ≤ 220:
Latitude: -33.92, Longitude: -57.92, Ozone Level: 178.10

OMI-Aura_L3-OMDOAO3e_2024m0113_v003-2024m0115t031948.he5
Found 3 data points in Uruguay with ozone levels ≤ 220:
Latitude: -34.67, Longitude: -54.66, Ozone Level: 132.20
Latitude: -30.42, Longitude: -54.91, Ozone Level: 216.30
Latitude: -30.17, Longitude: -54.91, Ozone Level: 216.30

OMI-Aura_L3-OMDOAO3e_2024m0120_v003-2024m0122t032324.he5
Found 6 data points in Uruguay with ozone levels ≤ 220:
Latitude: -30.67, Longitude: -55.91, Ozone Level: 199.20
Latitude: -30.67, Longitude: -55.41, Ozone Level: 26.60
Latitude: -30.42, Longitude: -56.41, Ozone Level: 150.80
Latitude: -30.42, Longitude: -56.16, Ozone Level: 150.80
Latitude: -30.42, Longitude: -55.91, Ozone Level: 204.00
Latitude: -30.42, Longitude: -55.66, Ozone Level: 204.00

OMI-Aura_L3-OMDOAO3e_2024m0125_v003-2024m0127t010754.he5
Found 3 data points in Uruguay w

OMI-Aura_L3-OMDOAO3e_2024m0409_v003-2024m0412t033421.he5
Found 6 data points in Uruguay with ozone levels ≤ 220:
Latitude: -31.67, Longitude: -57.16, Ozone Level: 207.20
Latitude: -31.67, Longitude: -53.91, Ozone Level: 219.40
Latitude: -31.67, Longitude: -53.66, Ozone Level: 219.40
Latitude: -30.92, Longitude: -55.66, Ozone Level: 219.10
Latitude: -30.92, Longitude: -55.41, Ozone Level: 219.10
Latitude: -30.42, Longitude: -56.91, Ozone Level: 216.10

OMI-Aura_L3-OMDOAO3e_2024m0411_v003-2024m0413t074653.he5
Found 2 data points in Uruguay with ozone levels ≤ 220:
Latitude: -30.67, Longitude: -53.16, Ozone Level: 214.20
Latitude: -30.42, Longitude: -53.16, Ozone Level: 208.30

OMI-Aura_L3-OMDOAO3e_2024m0413_v003-2024m0415t024820.he5
Found 2 data points in Uruguay with ozone levels ≤ 220:
Latitude: -34.67, Longitude: -55.16, Ozone Level: 178.60
Latitude: -34.67, Longitude: -54.91, Ozone Level: 178.60

OMI-Aura_L3-OMDOAO3e_2024m0415_v003-2024m0417t033858.he5
Found 4 data points in Uruguay 

C:\Users\emanu\AppData\Local\Temp\ipykernel_32632\283460377.py:22: RuntimeWarning: All-NaN slice encountered
  min_ozone_value = np.nanmin(uruguay_data)
C:\Users\emanu\AppData\Local\Temp\ipykernel_32632\283460377.py:23: RuntimeWarning: All-NaN slice encountered
  max_ozone_value = np.nanmax(uruguay_data)


An error occurred: index 0 is out of bounds for axis 0 with size 0
An error occurred: index 0 is out of bounds for axis 0 with size 0
OMI-Aura_L3-OMDOAO3e_2024m0715_v003-2024m0717t022818.he5
Found 3 data points in Uruguay with ozone levels ≤ 220:
Latitude: -32.67, Longitude: -53.91, Ozone Level: 205.30
Latitude: -31.67, Longitude: -56.41, Ozone Level: 186.40
Latitude: -31.42, Longitude: -56.41, Ozone Level: 206.80

OMI-Aura_L3-OMDOAO3e_2024m0719_v003-2024m0721t025042.he5
Found 3 data points in Uruguay with ozone levels ≤ 220:
Latitude: -30.42, Longitude: -55.66, Ozone Level: 210.70
Latitude: -30.42, Longitude: -55.41, Ozone Level: 210.70
Latitude: -30.17, Longitude: -55.41, Ozone Level: 210.70

OMI-Aura_L3-OMDOAO3e_2024m0727_v003-2024m0729t012924.he5
Found 2 data points in Uruguay with ozone levels ≤ 220:
Latitude: -33.17, Longitude: -55.66, Ozone Level: 192.40
Latitude: -33.17, Longitude: -55.41, Ozone Level: 192.40

OMI-Aura_L3-OMDOAO3e_2024m0802_v003-2024m0804t022541.he5
Found 4 dat

## Mapa

In [6]:
try:
    # Read the ozone data
    with h5py.File(file_path, "r") as file:
        ozone_data = file[dataset_path][:]
        print(f"Ozone data loaded with shape: {ozone_data.shape}")

    # Plot the ozone levels
    #plot_ozone_map_uruguay(ozone_data)
    plot_ozone_map(ozone_data)
    
except FileNotFoundError:
    print(f"File not found: {file_path}")
except KeyError:
    print(f"Dataset not found: {dataset_path}")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: Unable to synchronously open file (file signature not found)
